<a href="https://colab.research.google.com/github/benedettoscala/ifttt-code-generator/blob/main/test_and_compare_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install -U bitsandbytes

In [1]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

C:\Users\scala\PycharmProjects\JupyterProject\ifttt-code-generator\ifttt-code-generator


fatal: destination path 'ifttt-code-generator' already exists and is not an empty directory.
C:\Users\scala\miniconda3\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Already up to date.


### Model Comparison: GPT-2, BART, and Mistral for IFTTT Code Generation
This section evaluates and compares the performance of three fine-tuned language models (**GPT-2, BART, and Mistral**) in generating IFTTT-style automation code from textual descriptions.

#### **Dataset Loading and Preprocessing**
- The dataset is loaded from `"ifttt-code-generator/datasets/cleaned_and_combined.csv"`.
- The data is split into **80% training** and **20% testing**.
- **Test prompts** (natural language descriptions) and their corresponding **actual code** are extracted for evaluation.

#### **Model-Specific Generation Functions**
Each model uses a different approach for text generation:

1. **GPT-2 (`generate_with_gpt2`)**:
   - Loads a fine-tuned GPT-2 model from Hugging Face.
   - Encodes each test prompt and generates code using `generate()`, limiting output to 128 tokens.
   - The model is loaded onto a CUDA device for faster inference.

2. **BART (`generate_with_bart`)**:
   - Uses a **text-to-text generation pipeline**.
   - Each test prompt is formatted with an `"ifttt_prompt:"` prefix for consistent input format.
   - The generated text is extracted from the pipeline output.

3. **Mistral (`generate_with_mistral`)**:
   - Loads the **base model** `"Mistral-7B-Instruct-v0.2"` with **4-bit quantization** for memory efficiency.
   - Loads the **fine-tuned LoRA adapter** from `/content/drive/Shareddrives/NLPMODELS/mistral/checkpoint-20`.
   - Generates responses with:
     - `do_sample=True` (introduces randomness)
     - `top_k=50` (limits sampling to top 50 tokens)
     - `top_p=0.95` (nucleus sampling for diverse outputs)
     - `temperature=1` (controls creativity)

#### **Inference Execution**
- Each model generates code for all test prompts.
- Model-generated responses are stored in separate lists.

#### **Results Compilation**
- A `pandas` DataFrame is created containing:
  - **Prompt:** The original natural language description.
  - **Generated Code GPT-2:** The output from GPT-2.
  - **Generated Code BART:** The output from BART.
  - **Generated Code Mistral:** The output from Mistral.
  - **Actual Code:** The ground truth for comparison.

This setup enables **direct performance comparison** between the three models, helping assess which model best converts natural language descriptions into automation code.


In [1]:
import pandas as pd
import torch
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from peft import PeftModel

# Load the dataset and split it
df = pd.read_csv("datasets/new_dataset.csv")
#drop duplicates and null
df = df.drop_duplicates()
df = df.dropna()

train_df, test_df = train_test_split(df, test_size=0.356, random_state=42)

# Extract test set prompts
prompts = test_df["permission_df"].tolist()
actual_codes = test_df["filter_code"].tolist()

In [4]:
!pip show transformers



Name: transformers
Version: 4.49.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\scala\miniconda3\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


In [2]:


# Function to generate text with GPT-2
def generate_with_gpt2(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
    generated_codes = []

    for prompt in prompts:
        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to("cuda")
        attention_mask = input_data.attention_mask.to("cuda")

        output_ids = model.generate(input_ids, attention_mask=attention_mask, num_return_sequences=1)
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_codes.append(generated_text)

    del model
    del tokenizer
    torch.cuda.empty_cache()

    return generated_codes

# Function to generate text with BART
def generate_with_bart(model_path):
    generator = pipeline("text2text-generation", model=model_path, tokenizer=model_path)
    generated_codes = [generator(f"ifttt_prompt: {prompt}")[0]["generated_text"] for prompt in prompts]

    del generator
    torch.cuda.empty_cache()

    return generated_codes

# Function to generate text with Mistral

def generate_with_casual_lm(finetuned_model_path, basemodel_path):
    if not os.path.exists("./offload"):
        os.makedirs("./offload")

    print("Caricamento del modello fine-tunato...")
    bnb_config = BitsAndBytesConfig(
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False
    )

    max_memory = {
        0: "8GiB",  # Limite  sulla GPU
        "cpu": "16GiB"  # Limite  sulla CPU
    }

    model = AutoModelForCausalLM.from_pretrained(
        basemodel_path,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        device_map="auto",
        offload_folder="./offload",
        max_memory=max_memory
    )

    model = PeftModel.from_pretrained(model, finetuned_model_path)
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)

    generated_codes = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1,
        )
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        generated_codes.append(decoded_outputs[0])

    del model
    del tokenizer
    torch.cuda.empty_cache()

    return generated_codes




In [8]:
!dir

 Il volume nell'unit… C non ha etichetta.
 Numero di serie del volume: A216-BB2E

 Directory di C:\Users\scala\PycharmProjects\JupyterProject\ifttt-code-generator

02/22/2025  12:28 PM    <DIR>          .
02/22/2025  12:21 PM    <DIR>          ..
02/21/2025  02:31 PM                83 .gitattributes
02/21/2025  08:15 PM             1,594 0.43.0
02/21/2025  02:31 PM           424,408 bart_nl2ifttt.ipynb
02/21/2025  02:31 PM    <DIR>          datasets
02/21/2025  02:31 PM           239,405 fine_tuning_mistral.ipynb
02/21/2025  02:31 PM            29,432 generated_codes.csv
02/21/2025  02:31 PM            95,728 gpt2-nl2ifttt.ipynb
02/22/2025  12:27 PM    <DIR>          ifttt-code-generator
02/22/2025  11:20 AM    <DIR>          NLPMODELS
02/22/2025  12:16 PM    <DIR>          offload
02/21/2025  02:31 PM           224,787 preprocessing_and_cleaning.ipynb
02/21/2025  02:31 PM             2,597 README.md
02/21/2025  04:14 PM    <DIR>          results
02/21/2025  02:31 PM           249,872 

In [9]:
model_deepseek_path = "NLPMODELS/deepseek-coder/checkpoint-225"
basemodel_deepseek_path = "deepseek-ai/deepseek-coder-6.7b-base"
generated_codes_deepseek = generate_with_casual_lm(model_deepseek_path, basemodel_deepseek_path)

Caricamento del modello fine-tunato...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


KeyboardInterrupt: 

In [ ]:
model_codegemma_path = "NLPMODELS/codegemma/checkpoint-225"
basemodel_codegemma_path = "google/codegemma-7b"
generated_codes_codegemma = generate_with_casual_lm(model_codegemma_path, basemodel_codegemma)

In [ ]:
model_codellama_path = "NLPMODELS/codellama/checkpoint-225"
basemode_codellama_path = "codellama/CodeLlama-7b-hf"
generated_codes_codellama = generate_with_casual_lm(model_codellama_path, basemode_codellama_path)

In [ ]:
# Generate with BART
model_bart_path = "/content/drive/Shareddrives/NLPMODELS/nl2sql_bart_final/checkpoint-340"
generated_codes_bart = generate_with_bart(model_bart_path)

In [ ]:


# Create a DataFrame with results
results_df = pd.DataFrame({
    "Prompt": prompts,
    "Generated Code codegemma": generated_codes_codegemma,
    "Generated Code codellama": generated_codes_codellama,
    "Generated Code deepseek": generated_codes_deepseek,
    "Generated Code Bart" : generated_codes_bartm
    "Actual Code": actual_codes
})

In [ ]:
#create csv of results_df
results_df.to_csv("results_df.csv", index=False)

In [14]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e8a397cd245743ad35376a2e0ea92bcfa985f1e5bad49cb1c60b5388c4583d19
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [11]:
results_df

,Prompt,Generated Code GPT-2,Generated Code BART,Generated Code Mistral,Actual Code
0,This applet will reward you 1p for every 10 me...,This applet will reward you 1p for every 10 me...,var distance = parseInt(Strava.newActivityByYo...,This applet will reward you 1p for every 10 me...,var distance = parseInt(Strava.newActivityByYo...
1,Enter a description of your meal and the numbe...,Enter a description of your meal and the numbe...,if (Evernote.newEating.DescriptionByYou.Text.i...,Enter a description of your meal and the numbe...,var values = DoNote.doNoteNewCommandCommon.Not...
2,Turn on Wemo Switch After Garage Door Opens (A...,Turn on Wemo Switch After Garage Door Opens (A...,var timeOfDay = Meta.currentUserTime.hour() ...,Turn on Wemo Switch After Garage Door Opens (A...,var hour = Meta.currentUserTime.hour() if ...
3,Turn on WeMo Smart Plug When Ring Detects Moti...,Turn on WeMo Smart Plug When Ring Detects Moti...,var timeOfDay = Meta.currentUserTime.hour() ...,Turn on WeMo Smart Plug When Ring Detects Moti...,var timeOfDay = Meta.currentUserTime.hour() if...
4,This applet will add an iOS reminder to drink ...,This applet will add an iOS reminder to drink ...,var timeOfDay = Meta.triggerTime.hour() if (...,This applet will add an iOS reminder to drink ...,"var reminderTime = Meta.triggerTime.add(2, 'h'..."
5,"We got you, Dallas: this Applet sends you a Te...","We got you, Dallas: this Applet sends you a Te...",var Hour = Meta.currentUserTime.hour() var Day...,"We got you, Dallas: this Applet sends you a Te...",var Hour = Meta.currentUserTime.hour() var Day...
6,facebook only text post,facebook only text post notification to line n...,var text = FacebookPages.newStatusMessageByPag...,facebook only text post between 8:00 PM and 7:...,Facebook.newStatusMessageByYou.From Facebook.n...
7,If doorbell rings beween 21h and 6h then toggl...,If doorbell rings beween 21h and 6h then toggl...,var timeOfDay = Meta.currentUserTime.hour(); ...,If doorbell rings beween 21h and 6h then toggl...,var TimeOfDay = Meta.currentUserTime.hour() i...
8,Report today's rainfall amount from your Weath...,Report today's rainfall amount from your Weath...,if(parseFloat(Weather.currentWeather[0].rainfa...,Report today's rainfall amount from your Weath...,Netro.reportWeather.setDate(Netatmo.rainTodayA...
9,Turn light on when arriving to an area between...,Turn light on when arriving to an area between...,var timeOfDay = Meta.currentUserTime.hour() i...,Turn light on when arriving to an area between...,var timeOfDay = Meta.currentUserTime.hour(); ...


In [12]:
import pandas as pd

# Assicurarsi che tutti i valori siano stringhe
results_df["Generated Code GPT-2"] = results_df["Generated Code GPT-2"].astype(str)
results_df["Generated Code Mistral"] = results_df["Generated Code Mistral"].astype(str)

# Funzione per rimuovere il prompt e gestire errori
def clean_code(text):
    if "###" in text:
        return text.split("###", 1)[-1].strip()
    return text.strip()  # Se non c'è "###", restituisce la stringa originale
    # Funzione per rimuovere il prompt e gestire errori

# Applicare la funzione alla colonna
results_df["Generated Code GPT-2"] = results_df["Generated Code GPT-2"].apply(clean_code)
results_df["Generated Code Mistral"] = results_df["Generated Code Mistral"].apply(clean_code)

In [13]:
results_df

,Prompt,Generated Code GPT-2,Generated Code BART,Generated Code Mistral,Actual Code
0,This applet will reward you 1p for every 10 me...,var distance = parseInt(Strava.newActivityByYo...,var distance = parseInt(Strava.newActivityByYo...,const distanceMeters = parseInt(Strava.newActi...,var distance = parseInt(Strava.newActivityByYo...
1,Enter a description of your meal and the numbe...,,if (Evernote.newEating.DescriptionByYou.Text.i...,var calories = Regex.replace(Monzo.potDepositA...,var values = DoNote.doNoteNewCommandCommon.Not...
2,Turn on Wemo Switch After Garage Door Opens (A...,var hour = Meta.triggerTime.hour() if (hour <...,var timeOfDay = Meta.currentUserTime.hour() ...,"if (Meta.currentUserTime < ""17:00"") { WemoWi...",var hour = Meta.currentUserTime.hour() if ...
3,Turn on WeMo Smart Plug When Ring Detects Moti...,var timeOfDay = Meta.currentUserTime.hour(); ...,var timeOfDay = Meta.currentUserTime.hour() ...,var timeOfDay = Meta.currentUserTime.hour() ...,var timeOfDay = Meta.currentUserTime.hour() if...
4,This applet will add an iOS reminder to drink ...,var timeOfDay = Meta.currentUserTime.hour() ...,var timeOfDay = Meta.triggerTime.hour() if (...,let coffeeAmount = parseFloat(Monzo.cardPurcha...,"var reminderTime = Meta.triggerTime.add(2, 'h'..."
5,"We got you, Dallas: this Applet sends you a Te...",var Hour = Meta.currentUserTime.hour() var Day...,var Hour = Meta.currentUserTime.hour() var Day...,var Hour = Meta.currentUserTime.hour() var Day...,var Hour = Meta.currentUserTime.hour() var Day...
6,facebook only text post,var Seoul = moment.tz(',var text = FacebookPages.newStatusMessageByPag...,var hour = Meta.triggerTime.hour() if (hour >...,Facebook.newStatusMessageByYou.From Facebook.n...
7,If doorbell rings beween 21h and 6h then toggl...,var timeOfDay = Meta.currentUserTime.hour() if,var timeOfDay = Meta.currentUserTime.hour(); ...,var timeOfDay = Meta.currentUserTime.hour() i...,var TimeOfDay = Meta.currentUserTime.hour() i...
8,Report today's rainfall amount from your Weath...,if (parseInt(Netro.rainYesterdayAmount.Measure...,if(parseFloat(Weather.currentWeather[0].rainfa...,let rainmm = parseFloat(Weatherflow.rainTodayM...,Netro.reportWeather.setDate(Netatmo.rainTodayA...
9,Turn light on when arriving to an area between...,var timeOfDay = Meta.currentUserTime,var timeOfDay = Meta.currentUserTime.hour() i...,var timeOfDay = Meta.currentUserTime.hour() v...,var timeOfDay = Meta.currentUserTime.hour(); ...


### Evaluation of Generated Code
This section defines a function to evaluate the quality of model-generated code using multiple text similarity metrics.

#### **Evaluation Metrics**
- **BLEU Score (Bilingual Evaluation Understudy):**
  - Measures n-gram precision by comparing generated code with actual code.
  - Uses `sentence_bleu()` from NLTK for sentence-level evaluation.

- **METEOR Score (Metric for Evaluation of Translation with Explicit ORdering):**
  - Considers stemming, synonyms, and word order.
  - Computed using `single_meteor_score()` from NLTK.

- **ROUGE Scores (Recall-Oriented Understudy for Gisting Evaluation):**
  - Measures recall-based overlap between generated and reference text.
  - Three variations are computed:
    - **ROUGE-1:** Unigram (single-word) overlap.
    - **ROUGE-2:** Bigram (two-word) overlap.
    - **ROUGE-L:** Measures longest common subsequence similarity.

#### **Evaluation Process**
- For each pair of generated and actual code snippets:
  - Text is **tokenized** by splitting into words.
  - **BLEU, METEOR, and ROUGE scores** are computed.
- The function stores individual scores for all test samples.
- The **average score** is computed for each metric across the entire dataset.

#### **Returned Results**
- `mean_bleu`: Average BLEU score over all test samples.
- `mean_meteor`: Average METEOR score.
- `mean_rouge_l`: Average ROUGE-L score.
- `mean_rouge_1`: Average ROUGE-1 score.
- `mean_rouge_2`: Average ROUGE-2 score.

This function enables a **comprehensive evaluation** of model-generated code, ensuring a robust comparison against ground truth data.


In [15]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score

def evaluate_generated_text(generated_codes, actual_codes):
    bleu_scores = []
    meteor_scores = []
    rouge_l_scores = []
    rouge_1_scores = []
    rouge_2_scores = []

    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    scorer_1 = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)
    scorer_2 = rouge_scorer.RougeScorer(["rouge2"], use_stemmer=True)

    for gen, ref in zip(generated_codes, actual_codes):
        gen_tokens = gen.split()
        ref_tokens = ref.split()

        # Calcolo BLEU (sentence-level)
        bleu = sentence_bleu([ref_tokens], gen_tokens)

        # Calcolo METEOR (sentence-level)
        meteor = single_meteor_score(ref_tokens, gen_tokens)

        # Calcolo ROUGE-L (f-measure)
        rouge_l = scorer.score(ref, gen)["rougeL"].fmeasure
        # calcolo ROUGE-1
        rouge_1 = scorer_1.score(ref, gen)["rouge1"].fmeasure
        # calcolo ROUGE-2
        rouge_2 = scorer_2.score(ref, gen)["rouge2"].fmeasure

        bleu_scores.append(bleu)
        meteor_scores.append(meteor)
        rouge_l_scores.append(rouge_l)
        rouge_1_scores.append(rouge_1)
        rouge_2_scores.append(rouge_2)

    # Media su tutte le frasi del dataset di test
    mean_bleu = sum(bleu_scores) / len(bleu_scores)
    mean_meteor = sum(meteor_scores) / len(meteor_scores)
    mean_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
    mean_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
    mean_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)

    return mean_bleu, mean_meteor, mean_rouge_l, mean_rouge_1, mean_rouge_2


In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True


#### **Results Compilation**
- A `pandas` DataFrame (`metrics_df`) is created to organize evaluation scores.
- The DataFrame has the following structure:
  - **Metric** → The evaluation metric name.
  - **GPT-2** → Scores from the GPT-2 model.
  - **BART** → Scores from the BART model.
  - **Mistral** → Scores from the Mistral model.

This setup allows for **direct comparison** of model performance across multiple evaluation metrics.


In [17]:
# Evaluate models
gpt2_scores = evaluate_generated_text(generated_codes_gpt2, actual_codes)
bart_scores = evaluate_generated_text(generated_codes_bart, actual_codes)
mistral_scores = evaluate_generated_text(generated_codes_mistral, actual_codes)

metrics_df = pd.DataFrame(
    {
        "Metric": ["BLEU", "METEOR", "ROUGE-L", "ROUGE-1", "ROUGE-2"],
        "GPT-2": gpt2_scores,
        "BART": bart_scores,
        "Mistral": mistral_scores
    }
)


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [18]:
metrics_df

,Metric,GPT-2,BART,Mistral
0,BLEU,0.033308,0.177501,0.040425
1,METEOR,0.124226,0.344752,0.229492
2,ROUGE-L,0.191389,0.478291,0.217493
3,ROUGE-1,0.248590,0.502479,0.266189
4,ROUGE-2,0.122618,0.350900,0.148022


# Evaluation of Generated Code using Perplexity

This section defines functions to evaluate the quality of **model-generated code** using the **perplexity metric**. It includes methods for different types of language models, such as:
- **Causal models (e.g., GPT-2)**
- **Fine-tuned LoRA models with 4-bit quantization**
- **Seq2Seq models (e.g., BART, T5)**

## Evaluation Metric

### **Perplexity (PPL)**
- Measures how well a **probability model** predicts a given text.
- Lower perplexity **(PPL ↓)** means the model is more confident and the generated text is more accurate.
- Computed as:

  \[
  PPL = e^mean loss
  \]

- The **loss** is calculated using **cross-entropy** between the predicted tokens and the actual code.

---

## Evaluation Functions

The functions below compute **perplexity** for different scenarios:

### **1. `compute_perplexity_causal_prompt_target`**
- Computes **perplexity for a causal model** (e.g., **GPT-2**).
- **Masks the prompt tokens**, so the loss is computed **only on the generated code**.
- **Steps**:
  1. Load **model** and **tokenizer**.
  2. Concatenate **prompt** and **code**.
  3. Mask the **prompt tokens** (`-100` in PyTorch prevents loss computation on masked tokens).
  4. Compute **loss** and return **exponential of mean loss (perplexity)**.

---

### **2. `compute_perplexity_causal_prompt_target_lora4bit`**
- Computes **perplexity for a LoRA fine-tuned model** (**4-bit quantization**).
- Uses a **base model** and loads a fine-tuned **Low-Rank Adaptation (LoRA)** model.
- Follows similar steps as function (1), but:
  - Uses **quantization-aware loading** (`bnb_4bit_quant_type="nf4"`).
  - Loads the **fine-tuned LoRA model** using `PeftModel`.

---

### **3. `compute_perplexity_seq2seq`**
- Computes **perplexity for a sequence-to-sequence (Seq2Seq) model** (**e.g., BART, T5**).
- Evaluates how well the model translates a **prompt into the corresponding code**.
- **Steps**:
  1. Load **Seq2Seq model** and **tokenizer**.
  2. Tokenize **prompt** (input) and **code** (output).
  3. Compute **loss** on the generated code.
  4. Return **exponential of mean loss (perplexity)**.

---

## Evaluation Process

For each pair of **prompt and generated code**:
1. **Tokenize** text.
2. **Compute loss**:
   - **For causal models** → Loss is computed **only on the code part** (prompt is masked).
   - **For Seq2Seq models** → Loss is computed **on the entire generated sequence**.
3. Convert **loss** to **perplexity**.
4. Store **individual scores**.

- If the perplexity value is **infinity (`inf`)**, it is set to `float('inf')`.

In [19]:
import os
import math
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel

def compute_perplexity_causal_prompt_target(model_path, prompts, codes, max_length=512):
    """
    Calcola la Perplexity 'prompt -> code' con un modello causale (es. GPT-2),
    mascherando i token del prompt in modo che la loss sia calcolata solo sul code.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    model.eval()

    losses = []
    for prompt, code in zip(prompts, codes):
        full_text = f"{prompt}\n{code}"
        inputs = tokenizer(full_text, max_length=max_length, truncation=True, return_tensors='pt')
        input_ids = inputs.input_ids.to(device)

        labels = input_ids.clone()
        prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
        labels[0, :len(prompt_ids)] = -100  # Maschera il prompt

        with torch.no_grad():
            loss = model(input_ids=input_ids, labels=labels).loss.item()
            losses.append(loss)

    del model, tokenizer
    torch.cuda.empty_cache()

    return math.exp(np.mean(losses)) if not math.isinf(np.mean(losses)) else float('inf')

def compute_perplexity_causal_prompt_target_lora4bit(finetuned_model_path, base_model_path, prompts, codes, max_length=512):
    """
    Calcola la Perplexity 'prompt -> code' per un modello causale con LoRA e quantizzazione 4-bit.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    os.makedirs("./offload", exist_ok=True)

    bnb_config = BitsAndBytesConfig(
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, quantization_config=bnb_config, device_map="auto", offload_folder="./offload"
    )
    model = PeftModel.from_pretrained(base_model, finetuned_model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)
    model.eval()

    losses = []
    for prompt, code in zip(prompts, codes):
        full_text = f"{prompt}\n{code}"
        inputs = tokenizer(full_text, max_length=max_length, truncation=True, return_tensors='pt')
        input_ids = inputs.input_ids.to(device)

        labels = input_ids.clone()
        prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
        labels[0, :len(prompt_ids)] = -100  # Maschera il prompt

        with torch.no_grad():
            loss = model(input_ids=input_ids, labels=labels).loss.item()
            losses.append(loss)

    del model, tokenizer, base_model
    torch.cuda.empty_cache()

    return math.exp(np.mean(losses)) if not math.isinf(np.mean(losses)) else float('inf')

def compute_perplexity_seq2seq(model_path, texts, actual_codes, max_length=512):
    """
    Computes perplexity for an encoder-decoder (seq2seq) model like BART
    over pairs of (prompt, code).
    """
    import math
    import torch
    import numpy as np
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    model.eval()

    losses = []
    # Ciclo in parallelo: prompt -> code
    for text, code in zip(texts, actual_codes):
        # Prompt tokenizzato
        inputs = tokenizer(
            text, truncation=True, max_length=max_length, return_tensors='pt'
        )
        input_ids = inputs.input_ids.to(device)

        # Codice come labels
        labels = tokenizer(
            code, truncation=True, max_length=max_length, return_tensors='pt'
        ).input_ids.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            losses.append(loss.item())

    mean_loss = np.mean(losses)
    ppl = math.exp(mean_loss) if not math.isinf(mean_loss) else float('inf')

    # Cleanup
    del model
    del tokenizer
    torch.cuda.empty_cache()

    return ppl



In [20]:
model_gpt2_path = "/content/drive/Shareddrives/NLPMODELS/gpt2model/checkpoint-340"
ppl_gpt2 = compute_perplexity_causal_prompt_target(model_gpt2_path, prompts, actual_codes)
print("GPT-2 PPL (prompt->code):", ppl_gpt2)


GPT-2 PPL (prompt->code): 5.0172523763369155


In [21]:
finetuned_model_path = "/content/drive/Shareddrives/NLPMODELS/mistral/checkpoint-20"
basemodel_path = "mistralai/Mistral-7B-Instruct-v0.2"

ppl_mistral = compute_perplexity_causal_prompt_target_lora4bit(
    finetuned_model_path=finetuned_model_path,
    base_model_path=basemodel_path,
    prompts=prompts,
    codes=actual_codes
)
print("Mistral PPL (prompt->code):", ppl_mistral)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Mistral PPL (prompt->code): 2.871973808755254


In [22]:
# For BART
model_bart_path = "/content/drive/Shareddrives/NLPMODELS/nl2sql_bart_final/checkpoint-340"
ppl_bart = compute_perplexity_seq2seq(model_bart_path, prompts, actual_codes)
print("BART Perplexity:", ppl_bart)


BART Perplexity: 5.251900849772053
